In [1]:
!pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=15054 sha256=9bcb33894ff047f52c8acfb8448341ee15b61e0f38cfdf78197bf476369eedac
  Stored in directory: /tmp/pip-ephem-wheel-cache-ubcmpgs9/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1


In [2]:
import torch 

device = torch.device("cuda:3")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_MEDIUM

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.invariant_train import InvariantTrainsetWrapper
invariant_trainset = InvariantTrainsetWrapper(trainset, trainset.group_partition)

In [6]:
from torch.optim import SGD
from spuco.models import MLP
from spuco.invariant_train import GroupDRO

model = MLP(3*28*28, len(classes)).to(device)
group_dro = GroupDRO(
    model=model,
    num_epochs=1,
    trainset=invariant_trainset,
    num_groups=invariant_trainset.num_groups,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [ ]:
group_dro.c

In [7]:
group_dro.train()

Epoch 0:   0%|          | 1/938 [00:02<43:54,  2.81s/batch, accuracy=9.375%, loss=0.328]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
from spuco.util import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy